In [1]:
import os
import csv
import time
import gensim
from gensim.models.callbacks import CallbackAny2Vec

csv.field_size_limit(1000000)

131072

In [ ]:
# create iterator for corpus training
# from gensim.test.utils import datapath
# from gensim import utils

# class MyCorpus(object):
#     """An interator that yields sentences (lists of str)."""

#     def __iter__(self):
#         corpus_path = datapath('lee_background.cor')
#         for line in open(corpus_path):
#             # assume there's one document per line, tokens separated by whitespace
#             yield utils.simple_preprocess(line)

In [2]:
print('Read in tracks...')
tracks = {}
with open('tracks.csv', "r", encoding='utf-8') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';')
    for row in spamreader:
        columns = str(row)[2:-2].split(';')
        tracks[columns[0]] = [columns[1] + ' - ' +
                              columns[2], columns[3]] # title - artist, url
print(f'Number of tracks: {len(tracks)}')

# print('Read in playlists...')
# playlists = []
# with open('playlists.csv', 'r', encoding='utf-8') as csvfile:
#     spamreader = csv.reader(csvfile, delimiter=';')
#     for row in spamreader:
#         columns = str(row)[2:-2].split(';')
#         # if len(columns) < 1000: #limit length of playlists?
#         playlist = []
#         for column in columns[2:]:
#             playlist.append(column)
#         playlists.append(playlist)
# print(f'Number of playlists: {len(playlists)}')

Read in tracks...
Number of tracks: 19817445


In [3]:

# hyperparameters

min_count = 10      # minimum number of occurences of a track in all playlists
window = 2          # length of sequences
embedding_dim = 100 # number of dimensions in hidden layer
batch_words = 10000 # number of tracks to process in each batch
# iter = 20           # number of iterations
sg = 1              # skip-gram (1) or Continuous Bag Of Words (0)

# ------------------------------------------------------------------------------

valid_examples = [
    '2NMgVh5qaPprKTEzFe3501', # The Police - Roxanne
    '3Ti0GdlrotgwsAVBBugv0I', # A Tribe Called Quest - Can I Kick It?
    '0nyrltZrQGAJMBZc1bYvuQ', # James Brown - Get Up Offa That Thing
    '4hy4fb5D1KL50b3sng9cjw', # Nirvana - Smells Like Teen Spirit
    '1P49MJhU5vzttesFxw3dOM', # Bob Marley & The Wailers - Three Little Birds
    '76GlO5H5RT6g7y0gev86Nk', # The Cure - Just Like Heaven
    '40tAOP3DPqmVD6L1h45Jp6', # Frank Sinatra - My Way
    '4IMvgp0WZqr9mRqpEvDKxI', # The Clash - Rock the Casbah
    '1iDcKYNvo6gglrOG6lvnHL', # The Rolling Stones - Sympathy For The Devil
    '5uvosCdMlFdTXhoazkTI5R', # The Doors - Light My Fire
    '15JINEqzVMv3SvJTAXAKED', # Eminem - Love The Way You Lie
    '69kOkLUCkxIZYexIgSG8rq', # Daft Punk - Get Lucky
    '6oVY50pmdXqLNVeK8bzomn', # John Coltrane - My Favorite Things
    '6ui6l3ZNvlrGQZArwo8195', # Sex Pistols - God Save The Queen
    '0YammaEkYSeo9vQYZ1OwS6', # David Guetta - Say My Name
    '4SHZsQIdS2N1E5yqvoXF8o'  # Andy Williams - Can't Take My Eyes Off You
]

# ------------------------------------------------------------------------------

class logger(CallbackAny2Vec):
    def __init__(self):
        print('Starting...')
        self.epoch = 0
        self.loss = 0

    def on_train_begin(self, model):
        self.start = time.time()
        print(f'starting training epoch {self.epoch}')

    def on_epoch_end(self, model):
        elapsed = time.time() - self.start
        print('#{}'.format(self.epoch), 'loss =',
              (model.get_latest_training_loss() - self.loss) / batch_words,
              'elapsed time =', elapsed // 60, 'minutes', elapsed % 60, 'seconds')
        self.epoch += 1
        self.loss = model.get_latest_training_loss()
        print('Saving model...')
        model.save('word2vec.model')
        # _model = gensim.models.Word2Vec.load('word2vec.model')
        # for track in valid_examples:
        #     similar = _model.wv.most_similar(positive=[track], topn=8)
        #     most_similar = ''
        #     for i in range(0, 8):
        #         most_similar = most_similar + '%s (%.2f)' % (tracks[similar[i][0]][0], similar[i][1]) + ', '
        #     print('  %s -> %s' % (tracks[track][0], most_similar))
        # print()
        # del _model

In [4]:
# assert gensim.models.Word2Vec.FAST_VERSION > -1
model = gensim.models.Word2Vec(sentences=playlists, size=embedding_dim,
                               min_count=min_count, window=window, iter=iter,
                               batch_words=batch_words, compute_loss=True, sg=True,
                               callbacks=[logger()],workers=4)
print(model)

# no multi:
#start time 11:24
# ep1 12:37

# multi:
#start time 12:59
# ep 0 1:30

Starting...
starting training epoch 0
#0 loss = 8115.2232 elapsed time = 33.0 minutes 5.925987005233765 seconds
Saving model...
#1 loss = 2720.6224 elapsed time = 71.0 minutes 27.31136202812195 seconds
Saving model...
#2 loss = 2585.9272 elapsed time = 109.0 minutes 57.78689408302307 seconds
Saving model...
#3 loss = 0.0 elapsed time = 147.0 minutes 48.547168016433716 seconds
Saving model...
#4 loss = 0.0 elapsed time = 184.0 minutes 0.7114369869232178 seconds
Saving model...


KeyboardInterrupt: 

In [5]:
# print(model)

NameError: name 'model' is not defined

In [17]:
# model.save('song2vec.model')

In [16]:
# model.wv.save_word2vec_format(fname='song2vec.bin',fvocab='songVocab.bin',binary=True)

In [4]:
import gensim

model = gensim.models.Word2Vec.load('word2vec.model')
print(model)

Word2Vec(vocab=2924687, size=100, alpha=0.025)


In [7]:
for track in valid_examples:
            similar = model.wv.most_similar(positive=[track], topn=8)
            most_similar = ''
            for i in range(0, 8):
                most_similar = most_similar + '%s (%.2f)' % (tracks[similar[i][0]][0], similar[i][1]) + ', '
            print('  %s -> %s' % (tracks[track][0], most_similar))

The Police - Roxanne -> Europe - The Final Countdown (0.85), The Police - Can't Stand Losing You (0.83), The Police - Walking On The Moon - 2003 Stereo Remastered Version (0.83), Men At Work - Down Under (0.83), The Police - Roxanne (0.83), The Beatles Recovered Band - Help! (0.82), Elvis Presley - A Little Less Conversation - JXL Radio Edit Remix (0.82), Peter Gabriel - Sledgehammer - Live (0.82), 
  A Tribe Called Quest - Can I Kick It? -> A Tribe Called Quest - Electric Relaxation (0.84), A Tribe Called Quest - Bonita Applebum - includes 'Can I Kick It' Intro (0.83), The Pharcyde - Passin' Me By (0.83), The Pharcyde - Runnin' (0.83), Souls Of Mischief - 93 'Til Infinity (0.83), The Pharcyde - Runnin' (0.82), Pete Rock & C.L. Smooth - They Reminisce Over You (T.R.O.Y.) (0.81), A Tribe Called Quest - Scenario (0.81), 
  James Brown - Get Up Offa That Thing -> James Brown - Papa's Got A Brand New Bag (0.81), James Brown - Get Up (I Feel Like Being A) Sex Machine - Pt. 1 & 2 (0.80), Jam